<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.32.1
    Uninstalling protobuf-6.32.1:


      Successfully uninstalled protobuf-6.32.1


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.66


    Uninstalling yfinance-0.2.66:
      Successfully uninstalled yfinance-0.2.66
   ━━━━━━━━━━━━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━ 1/2 [yfinance]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-09-26 09:15:43
-------------------
qualified stocks: 87
with latest results: 27
still star stocks: 16
-------------------
Initial Investment:  1.22 C
CY Investment:  1.49 C
Reserve:  4.38 K
Current:  1.38 C
-------------------
Today PnL: -35.53 K (-0.26%)
Current PnL: -21.41 L (-14.4%)
CY Booked + Current PnL: -9.97 L (-6.71%)
-------------------
Total profit:  1.15 L
Total loss:  -22.56 L
-------------------
Total Booked + Current PnL: 17.01 L (13.98%)
Total Booked PnL: 38.42 L (31.57%)
Curr Year Booked PnL: 11.44 L (8.28%)
Prev Year Booked PnL: 26.98 L (22.17%)
Est FTT:  2.25 C
Est FTT PnL: 87.28 L (63.16%)
Deployed:  1.22 C
Current:  1.38 C
CAGR/XIRR %: 7.72%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
77,TTKPRESTIG,770.00,97.65,43.0,M-SC,2.81,85150.0,-15627.0,15719.0,-0.49,-15.51,18.46,0.09,245.0,-0.99,0.62,11.87,OX40N,NTT,DURABLES
14,BLUESTARCO,2080.00,11.16,52.0,H-MC,3.87,191970.0,27300.0,16029.0,-0.39,16.58,8.35,26.31,89.0,1.70,1.39,25.88,X40N,NTT,AC
50,NATIONALUM,244.55,-45.32,55.0,H-MC,11.02,109069.0,8565.0,20538.0,0.09,8.52,18.83,28.96,79.0,0.42,0.79,45.38,MH,ATH,METALS
84,VOLTAS,1530.00,-2.20,45.0,H-MC,3.07,205320.0,13578.0,24187.0,-0.26,7.08,11.78,19.69,99.0,0.56,1.49,14.36,XY25,NTT,AC
42,ITC,452.00,-39.19,38.0,H-LC,2.20,195722.0,-4416.0,25307.0,0.04,-2.21,12.93,10.44,4.0,-0.17,1.42,3.23,X40,NTT,FMCG


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
32,HINDZINC,730.22,24.68,63.0,M-LC,2.47,203036.0,-2040.0,114614.0,3.06,-0.99,56.45,54.89,52.0,-0.02,1.47,22.86,X5K,ATH,METALS
7,ATULAUTO,844.00,3656.76,51.0,M-SC,7.17,117032.0,-30336.0,82156.0,0.85,-20.59,70.20,35.16,236.0,-0.37,0.85,19.54,XY24,NTT,AUTO
12,BATAINDIA,2096.00,-33.72,47.0,M-SC,4.97,99094.0,-29576.0,74875.0,0.73,-22.99,75.56,35.20,219.0,-0.40,0.72,13.74,X40,NTT,FOOTWEAR
67,SONACOMS,1006.00,-42.11,34.0,M-SC,12.29,71995.0,-29164.0,104054.0,0.43,-28.83,144.53,74.03,202.0,-0.28,0.52,1.52,AR,BTT,AUTO
66,SIS,528.00,2092.23,42.0,H-SC,4.24,88472.0,-22560.0,46165.0,0.25,-20.32,52.18,21.26,156.0,-0.49,0.64,19.53,OX40N,NTT,MISC


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
82,VALIANTORG,838.0,-305.86,37.0,H-SC,4.20,114342.0,-49263.0,170575.0,-1.95,-30.11,149.18,74.15,139.0,-0.29,0.83,46.57,XR,NTT,CHEMICALS
19,CIPLA,1795.0,-22.52,39.0,H-LC,4.86,206829.0,2329.0,39091.0,-1.84,1.14,18.90,20.25,10.0,0.06,1.50,9.97,X40N,BTT,PHARMA
26,GREENPANEL,537.0,206.85,46.0,M-SC,0.85,137006.0,-46072.0,125045.0,-1.77,-25.17,91.27,43.14,230.0,-0.37,0.99,28.87,XY24,NTT,MISC
78,UJJIVANSFB,60.0,90.20,46.0,H-SC,10.13,119961.0,-22518.0,42034.0,-1.20,-15.80,35.04,13.70,163.0,-0.54,0.87,41.59,OX40N,NTT,BANKS
61,SAMMAANCAP,326.0,-170.45,52.0,M-SC,5.82,80922.0,-21288.0,114675.0,-1.13,-20.83,141.71,91.37,208.0,-0.19,0.59,31.36,XY25,NTT,FINANCE


In [12]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
23,DABUR,735.00,-9.24,35.0,H-MC,6.62,245798.0,-2270.0,110683.0,0.10,-0.91,45.03,43.70,102.0,-0.02,1.78,13.40,XY24,BTT,FMCG
32,HINDZINC,730.22,24.68,63.0,M-LC,2.47,203036.0,-2040.0,114614.0,3.06,-0.99,56.45,54.89,52.0,-0.02,1.47,22.86,X5K,ATH,METALS


In [13]:
# OX40N stocks to exit
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
77,TTKPRESTIG,770.0,97.65,43.0,M-SC,2.81,85150.0,-15627.0,15719.0,-0.49,-15.51,18.46,0.09,245.0,-0.99,0.62,11.87,OX40N,NTT,DURABLES
78,UJJIVANSFB,60.0,90.20,46.0,H-SC,10.13,119961.0,-22518.0,42034.0,-1.20,-15.80,35.04,13.70,163.0,-0.54,0.87,41.59,OX40N,NTT,BANKS
45,KANSAINER,340.0,-67.97,50.0,H-SC,1.41,220626.0,-49041.0,85382.0,-0.04,-18.19,38.70,13.47,138.0,-0.57,1.60,11.59,XY24,NTT,PAINTS
18,CERA,9475.0,-22.91,33.0,H-SC,1.86,140369.0,-35534.0,77554.0,-0.41,-20.20,55.25,23.89,149.0,-0.46,1.02,21.23,OX40N,NTT,CERAMICS
66,SIS,528.0,2092.23,42.0,H-SC,4.24,88472.0,-22560.0,46165.0,0.25,-20.32,52.18,21.26,156.0,-0.49,0.64,19.53,OX40N,NTT,MISC


In [14]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [15]:
# top 5 to monitor for exit
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
60,SAIL,228.00,49.16,68.0,M-MC,10.41,235242.0,10280.0,157377.0,0.07,4.57,66.90,74.53,192.0,0.07,1.70,38.30,XY24,BTT,STEEL
14,BLUESTARCO,2080.00,11.16,52.0,H-MC,3.87,191970.0,27300.0,16029.0,-0.39,16.58,8.35,26.31,89.0,1.70,1.39,25.88,X40N,NTT,AC
84,VOLTAS,1530.00,-2.20,45.0,H-MC,3.07,205320.0,13578.0,24187.0,-0.26,7.08,11.78,19.69,99.0,0.56,1.49,14.36,XY25,NTT,AC
17,CAMS,5211.76,-4.94,46.0,H-SC,1.75,108024.0,6020.0,37906.0,-0.27,5.90,35.09,43.06,122.0,0.16,0.78,24.03,X40N,ATH,MISC
79,UNIONBANK,163.00,-13.18,58.0,M-LC,6.86,156644.0,15804.0,28682.0,-0.72,11.22,18.31,31.59,66.0,0.55,1.13,40.68,XY24,NTT,BANKS


In [16]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
50,NATIONALUM,244.55,-45.32,55.0,H-MC,11.02,109069.0,8565.0,20538.0,0.09,8.52,18.83,28.96,79.0,0.42,0.79,45.38,MH,ATH,METALS
11,BANKINDIA,190.00,-29.19,58.0,H-MC,8.26,183283.0,3475.0,108943.0,-0.49,1.93,59.44,62.52,88.0,0.03,1.33,36.68,XR,NTT,BANKS
85,WHIRLPOOL,2270.00,-44.58,32.0,M-SC,1.49,93660.0,2162.0,76586.0,-0.02,2.36,81.77,86.07,223.0,0.03,0.68,36.01,XR,NTT,DURABLES
37,INDIAMART,4850.92,-55.20,25.0,H-SC,10.23,125234.0,1898.0,131871.0,-0.45,1.54,105.30,108.45,119.0,0.01,0.91,24.24,AR,ATH,MISC
86,WIPRO,420.00,-15.15,37.0,M-LC,5.61,149358.0,-1587.0,111048.0,-0.45,-1.05,74.35,72.51,53.0,-0.01,1.08,4.91,XR,NTT,IT


In [17]:
# top 5 to accumulate based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
5,ASIANPAINT,3465.66,-16.59,32.0,H-LC,7.11,210531.0,-41237.0,94444.0,-0.48,-16.38,44.86,21.13,27.0,-0.44,1.52,13.24,X40,ATH,PAINTS
44,JIOFIN,387.00,-4.63,32.0,H-LC,12.98,259032.0,-8333.0,73798.0,-0.13,-3.12,28.49,24.48,48.0,-0.11,1.88,50.14,XY24,BTT,FINANCE
74,TCS,4476.75,-30.61,32.0,H-LC,12.05,276877.0,-69079.0,143948.0,-0.40,-19.97,51.99,21.64,1.0,-0.48,2.00,0.00,X40,ATH,IT
83,VBL,671.64,-20.19,32.0,H-LC,8.17,287451.0,-28391.0,141052.0,-0.29,-8.99,49.07,35.67,5.0,-0.20,2.08,3.62,X40N,ATH,FMCG
42,ITC,452.00,-39.19,38.0,H-LC,2.20,195722.0,-4416.0,25307.0,0.04,-2.21,12.93,10.44,4.0,-0.17,1.42,3.23,X40,NTT,FMCG


In [18]:
# top 5 to accumulate based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
48,LTIM,7230.2,-5.49,47.0,H-LC,1.67,234630.0,-11521.0,90731.0,0.13,-4.68,38.67,32.18,16.0,-0.13,1.70,29.68,X200,ATH,IT
42,ITC,452.0,-39.19,38.0,H-LC,2.20,195722.0,-4416.0,25307.0,0.04,-2.21,12.93,10.44,4.0,-0.17,1.42,3.23,X40,NTT,FMCG
65,SIEMENS,4671.5,-2.40,44.0,H-LC,2.61,156750.0,-29345.0,76823.0,-0.67,-15.77,49.01,25.51,15.0,-0.38,1.13,15.51,AR,ATH,ELECTRICAL
57,RELIANCE,1533.0,-14.79,41.0,H-LC,4.06,213751.0,3385.0,25394.0,-0.16,1.61,11.88,13.68,37.0,0.13,1.55,18.40,XY25,NTT,REFINERIES
51,NESTLEIND,1377.0,-10.90,45.0,H-LC,4.37,274318.0,8892.0,47896.0,-0.10,3.35,17.46,21.40,11.0,0.19,1.99,10.93,XY25,NTT,FMCG


In [19]:
# top 5 to accumulate based on lowest Gained%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'Gained%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
74,TCS,4476.75,-30.61,32.0,H-LC,12.05,276877.0,-69079.0,143948.0,-0.40,-19.97,51.99,21.64,1.0,-0.48,2.00,0.00,X40,ATH,IT
80,UNITDSPR,1644.00,-14.09,45.0,H-LC,7.16,225165.0,-8979.0,57597.0,-0.65,-3.83,25.58,20.77,86.0,-0.16,1.63,2.82,X40N,NTT,BREWERIES
42,ITC,452.00,-39.19,38.0,H-LC,2.20,195722.0,-4416.0,25307.0,0.04,-2.21,12.93,10.44,4.0,-0.17,1.42,3.23,X40,NTT,FMCG
83,VBL,671.64,-20.19,32.0,H-LC,8.17,287451.0,-28391.0,141052.0,-0.29,-8.99,49.07,35.67,5.0,-0.20,2.08,3.62,X40N,ATH,FMCG
30,HCLTECH,1943.91,-2.80,38.0,H-LC,9.89,222186.0,-19738.0,83009.0,-0.60,-8.16,37.36,26.15,8.0,-0.24,1.61,5.00,X40,ATH,IT


In [20]:
# top 5 to accumulate based on lowest CurrAlloc%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
65,SIEMENS,4671.50,-2.40,44.0,H-LC,2.61,156750.0,-29345.0,76823.0,-0.67,-15.77,49.01,25.51,15.0,-0.38,1.13,15.51,AR,ATH,ELECTRICAL
42,ITC,452.00,-39.19,38.0,H-LC,2.20,195722.0,-4416.0,25307.0,0.04,-2.21,12.93,10.44,4.0,-0.17,1.42,3.23,X40,NTT,FMCG
19,CIPLA,1795.00,-22.52,39.0,H-LC,4.86,206829.0,2329.0,39091.0,-1.84,1.14,18.90,20.25,10.0,0.06,1.50,9.97,X40N,BTT,PHARMA
5,ASIANPAINT,3465.66,-16.59,32.0,H-LC,7.11,210531.0,-41237.0,94444.0,-0.48,-16.38,44.86,21.13,27.0,-0.44,1.52,13.24,X40,ATH,PAINTS
57,RELIANCE,1533.00,-14.79,41.0,H-LC,4.06,213751.0,3385.0,25394.0,-0.16,1.61,11.88,13.68,37.0,0.13,1.55,18.40,XY25,NTT,REFINERIES


In [21]:
# top 5 to accumulate based on CumlRnk
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
74,TCS,4476.75,-30.61,32.0,H-LC,12.05,276877.0,-69079.0,143948.0,-0.40,-19.97,51.99,21.64,1.0,-0.48,2.00,0.00,X40,ATH,IT
40,INFY,2275.00,-19.24,44.0,H-LC,8.10,314218.0,1330.0,170369.0,-0.65,0.42,54.22,54.87,3.0,0.01,2.27,7.10,X40,BTT,IT
42,ITC,452.00,-39.19,38.0,H-LC,2.20,195722.0,-4416.0,25307.0,0.04,-2.21,12.93,10.44,4.0,-0.17,1.42,3.23,X40,NTT,FMCG
83,VBL,671.64,-20.19,32.0,H-LC,8.17,287451.0,-28391.0,141052.0,-0.29,-8.99,49.07,35.67,5.0,-0.20,2.08,3.62,X40N,ATH,FMCG
1,ABB,7934.00,-40.25,45.0,H-LC,7.94,249600.0,-12019.0,131240.0,-0.01,-4.59,52.58,45.57,7.0,-0.09,1.81,6.21,AR,NTT,ELECTRICAL


In [22]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
21,COFFEEDAY,80.00,-45.67,45.0,L-SC,41.52,81024.0,-32525.0,72573.0,-0.40,-28.64,89.57,35.27,268.0,-0.45,0.59,98.31,XR,NTT,HOTELS
49,MASFIN,398.61,-18.70,42.0,H-SC,12.55,92220.0,-5760.0,27362.0,-0.45,-5.88,29.67,22.05,152.0,-0.21,0.67,34.05,XR,ATH,FINANCE
50,NATIONALUM,244.55,-45.32,55.0,H-MC,11.02,109069.0,8565.0,20538.0,0.09,8.52,18.83,28.96,79.0,0.42,0.79,45.38,MH,ATH,METALS
78,UJJIVANSFB,60.00,90.20,46.0,H-SC,10.13,119961.0,-22518.0,42034.0,-1.20,-15.80,35.04,13.70,163.0,-0.54,0.87,41.59,OX40N,NTT,BANKS
79,UNIONBANK,163.00,-13.18,58.0,M-LC,6.86,156644.0,15804.0,28682.0,-0.72,11.22,18.31,31.59,66.0,0.55,1.13,40.68,XY24,NTT,BANKS


In [23]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
64,SHALBY,327.00,1175.92,65.0,M-SC,12.72,173596.0,-7547.0,53016.0,-0.20,-4.17,30.54,25.10,235.0,-0.14,1.26,39.12,XY24,NTT,HEALTHCARE
71,TANLA,1963.11,-17.86,60.0,H-SC,16.92,220014.0,-50747.0,380690.0,-0.23,-18.74,173.03,121.86,133.0,-0.13,1.59,72.11,AR,ATH,IT
81,VAIBHAVGBL,521.00,55.37,53.0,H-SC,5.60,134875.0,-47900.0,158964.0,-0.78,-26.21,117.86,60.77,125.0,-0.30,0.98,20.80,XR,NTT,JEWELLERY
7,ATULAUTO,844.00,3656.76,51.0,M-SC,7.17,117032.0,-30336.0,82156.0,0.85,-20.59,70.20,35.16,236.0,-0.37,0.85,19.54,XY24,NTT,AUTO
60,SAIL,228.00,49.16,68.0,M-MC,10.41,235242.0,10280.0,157377.0,0.07,4.57,66.90,74.53,192.0,0.07,1.70,38.30,XY24,BTT,STEEL


In [24]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,19.24
1,25,43.73
2,50,74.90


In [25]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    43.91
LC    32.41
MC    23.73
Name: CurrAlloc%, dtype: float64

In [26]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     30.35
X40      14.75
X40N     11.38
XR       11.17
XY25     10.15
AR        8.34
OX40N     7.87
X200      1.70
MH        1.65
X5K       1.47
SR        1.22
Name: CurrAlloc%, dtype: float64

In [27]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-SC    29.37
H-LC    25.93
H-MC    20.74
M-SC    13.05
M-LC     5.43
M-MC     2.68
L-SC     1.49
L-LC     1.05
L-MC     0.31
Name: CurrAlloc%, dtype: float64

In [28]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,14.05,-5.85,40.78
IT,12.91,-17.79,82.02
FINANCE,10.05,-18.30,69.91
MISC,6.99,-18.87,84.95
BANKS,6.15,-17.59,79.44
PAINTS,5.66,-18.08,35.68
ELECTRICAL,5.46,-9.06,47.33
INSURANCE,3.77,-6.21,45.91
AC,3.58,2.09,14.31


In [29]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,3190202.0,22
XR,1370243.0,14
AR,1180649.0,9
X40,810133.0,10
X40N,632801.0,9
OX40N,559978.0,10
XY25,440135.0,7
SR,252231.0,2
X5K,114614.0,1


In [30]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3738582.0,29
M-SC,1444434.0,17
H-MC,1242700.0,15
H-LC,1238990.0,15
M-LC,399800.0,4
M-MC,307116.0,2
L-SC,256508.0,3
L-MC,60467.0,1
L-LC,39286.0,1


In [31]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1244008.0      6
M-SC       XY24       824396.0      7
H-SC       AR         803411.0      5
           XR         796705.0      7
H-MC       XY24       566746.0      4
H-LC       X40        517077.0      5
H-SC       X40N       312042.0      4
M-MC       XY24       307116.0      2
H-SC       OX40N      264557.0      4
           SR         252231.0      2
H-LC       X40N       237740.0      3
H-MC       X40        218181.0      4
H-LC       AR         208063.0      2
H-MC       XY25       174593.0      2
M-LC       XY24       174138.0      2
M-SC       AR         169175.0      2
L-SC       XR         169094.0      2
M-SC       OX40N      137327.0      4
           XR         123986.0      2
           XY25       114675.0      1
M-LC       X5K        114614.0      1
H-LC       XY25       111581.0      3
M-LC       XR         111048.0      1
H-MC       XR         108943.0      1
H-LC       X200        90731.0      1
L-SC       OX40N       87414.0      1
H-MC       X40N        83019.0      2
M-SC       X40         74875.0      1
H-LC       XY24        73798.0      1
H-MC       OX40N       70680.0      1
H-SC       MH          65628.0      1
L-MC       XR          60467.0      1
L-LC       XY25        39286.0      1
H-MC       MH          20538.0      1

In [32]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 37.0 seconds
